In [1]:
# Pitch Recognition Forked From https://github.com/justinsalamon/audio_to_midi_melodia
import os
import numpy as np
import pretty_midi as midi
import pitch_rec

In [2]:
infile = "testo.mp3"
outfile = "melody.mid"
bpm = 120
high_mapper = np.array([67, 69, 71, 72, 74, 76])
male_mapper = np.array([60, 68, 67, 64, 62, 60])
female_mapper = np.array([69, 62, 63, 60, 67, 69])

In [3]:
pitch_rec.audio_to_midi_melodia(infile, outfile, bpm, hop=128, smooth=0.1)

Loading audio...
Extracting melody f0 with MELODIA...
Converting Hz to MIDI notes...
Saving MIDI to disk...
Conversion complete.


In [4]:
data = midi.PrettyMIDI("./melody.mid")
pitches = list(map(lambda x: x.pitch, data.instruments[0].notes))
high_pitch = high_mapper[high_mapper <= max(pitches)][-1]
idx = np.where(high_mapper == high_pitch)[0][0]
pitch = female_mapper[idx]
print(pitch)

60
